In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# データの読み込みと前処理を行う関数
def load_and_process_data(file_path):
    # Excelファイルを読み込む
    df = pd.read_excel(file_path, sheet_name='売上データ')

    # 「売上」列を追加（単価 * 数量）
    df["売上"] = df["単価"] * df["数量"]

    # 「売上日」列をdatetime型に変換
    df["売上日"] = pd.to_datetime(df["売上日"])

    # 「年月」列を追加（売上日の年と月を抽出）
    df["年月"] = df["売上日"].dt.strftime('%Y-%m')

    # データフレームを返す
    return df

In [3]:
# 集計を行う関数
def sum_sales_data(df):
    # 月ごとのカテゴリー別売上合計を集計
    pivot_table = pd.pivot_table(
        df,
        index='カテゴリー',   # 行にする項目
        columns='年月',       # 列にする項目
        values='売上',        # 集計対象の項目
        aggfunc='sum',        # 集計方法（ここでは合計を指定）
        fill_value=0          # NaNの代わりに埋める値
    )

    # ピボットテーブルを返す
    return pivot_table

In [4]:
# データをプロンプトに変換する関数
def convert_to_prompt(df, pivot_table):
    # データフレーム全体を文字列に変換
    sales_data_text = df.astype(str)

    # 集計結果を文字列に変換
    pivot_table_text = pivot_table.astype(str)

    # プロンプトの作成
    prompt_text = f"""
    売上データ:
    {sales_data_text}
    月ごとのカテゴリー別売上合計：
    {pivot_table_text}
    上記の「売上データ」と「月ごとのカテゴリー別売上合計」をもとに、カテゴリー毎の売上戦略を考案してください。
    """

    # プロンプトを返す
    return prompt_text

In [5]:
# OpenAI APIの呼び出しを行う関数
def get_openai_response(client, prompt_text, model_name="gpt-4o-mini"):
    # 役割を設定
    role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

    # APIへリクエスト
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt_text},
        ],
    )

    # 結果を取得
    result = response.choices[0].message.content.strip()

    # 結果を返す
    return result

In [6]:
# 結果をファイルに保存する関数
def save_result_to_file(result, file_path="カテゴリー毎の売上戦略.md"):
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(result)

In [7]:
# ワークフロー（メイン処理）
def main():
    print("処理を開始します。")

    # 1.データの読み込みと前処理
    print("（1/5）データの読み込みと前処理")
    df = load_and_process_data('サンプルデータ.xlsx')

    # 2.集計
    print("（2/5）集計")
    pivot_table = sum_sales_data(df)

    # 3.プロンプト生成
    print("（3/5）プロンプト生成")
    prompt_text = convert_to_prompt(df, pivot_table)

    # 4.OpenAI APIからの応答を取得
    print("（4/5）OpenAI APIからの応答を取得")
    result = get_openai_response(client, prompt_text)

    # 5.結果をファイルに保存
    print("（5/5）結果をファイルに保存")
    save_result_to_file(result)
    print("分析結果を保存しました。")

In [8]:
# ワークフローの実行
main()

処理を開始します。
（1/5）データの読み込みと前処理
（2/5）集計
（3/5）プロンプト生成
（4/5）OpenAI APIからの応答を取得
（5/5）結果をファイルに保存
分析結果を保存しました。
